In [1]:
# PREDICTION WITH TRAINED ANN MODEL
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# load the trained model, scaler, pickle, onehot
model=load_model('model.h5')

# load the encoders and scaler
with open('onehot_encoder_geo.pk1', 'rb')as file:
    label_encoder_geo=pickle.load(file)
    
with open('label_encoder_gender.pk1', 'rb')as file:
    label_encoder_gender=pickle.load(file)
    
with open('scaler.pk1', 'rb')as file:
    scaler=pickle.load(file)

In [3]:
# Example input data
input_data={
    'CreditScore':50000,
    'Geography':'France',
    'Gender':'Female',
    'Age':40,
    'Tenure':4,
    'Balance':60000,
    'NumOfProducts':3,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [4]:
# One-hot encode 'Geography'
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\Project_work\deep_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
# Here out input data is in the key_value format, so we have to convert it into dataframe before use
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,50000,France,Female,40,4,60000,3,1,1,50000


In [6]:
# #Encode categorical Variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,50000,France,0,40,4,60000,3,1,1,50000


In [7]:
# Concatination of one hot encoded
input_df=pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,50000,0,40,4,60000,3,1,1,50000,1.0,0.0,0.0


In [8]:
# Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 5.11751725e+02, -1.09456700e+00,  1.05139783e-01,
        -3.41719044e-01, -2.61085150e-01,  2.54529536e+00,
         6.45062939e-01,  9.69348422e-01, -8.79473428e-01,
         1.00262813e+00, -5.79611284e-01, -5.77109737e-01]])

In [9]:
# Prediction or predict the churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


array([[0.]], dtype=float32)

In [10]:
# probaility of the prediction
prediction_prob= prediction[0][0]
prediction_prob

np.float32(0.0)

In [11]:
if prediction_prob>0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn")
    

The customer is not likely to churn
